# pγ→ηπ⁰p amplitude model in symbolic expression

Formulate Helicity Amplitude Model for pγ→ηπ⁰p symbolically

## Generate Transitions

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
from ampform.io import aslatex, improve_latex_rendering
from IPython.display import Math
from qrules.particle import load_pdg
from qrules.particle import Particle, Spin
from ampform.dynamics.builder import RelativisticBreitWignerBuilder
from tensorwaves.data import TFPhaseSpaceGenerator, TFUniformRealNumberGenerator
from tensorwaves.function.sympy import create_parametrized_function
from tensorwaves.data import SympyDataTransformer

import graphviz
import qrules
import ampform
import numpy as np
import pandas as pd

improve_latex_rendering()
particle_db = load_pdg()

In [ ]:
pgamma = Particle(
    name="pgamma",
    latex=r"p\gamma (s1/2)",
    spin=0.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99990,
)

In [ ]:
pgamma2 = Particle(
    name="pgamma2",
    latex=R"p\gamma (s3/2)",
    spin=1.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99991,
)

In [ ]:
pgamma_states = [pgamma, pgamma2]

In [ ]:
particle_db.update(pgamma_states)

In [ ]:
reaction_strong_EM = qrules.generate_transitions(
    initial_state="pgamma",
    final_state=["eta", "pi0", "p"],
    allowed_interaction_types=["strong", "EM"],
    formalism="canonical-helicity",
    particle_db=particle_db,
    max_angular_momentum=3,
    max_spin_magnitude=3,
    mass_conservation_factor=0,
)

In [ ]:
src = qrules.io.asdot(reaction_strong_EM, collapse_graphs=True)
graphviz.Source(src)

We keep the relevant information about branching fractions in the previous [Branching fraction](docs/branching-fraction.md) chapter.
For simplicity, after the information about branching fraction, and under the current `qrules` results, we limit the intermediate resonances to be:

"$a(2)$" for $\eta \pi^0$, 
"$\Delta(1232)$ for $\pi p$"
Both "$N(1535)$" for $\pi^0 p$
and $\eta p$

In [ ]:
reaction_strong_EM = qrules.generate_transitions(
    initial_state="pgamma",
    final_state=["eta", "pi0", "p"],
    allowed_intermediate_particles=["a(2)(1320)", "N(1535)", "Delta(1232)"],
    allowed_interaction_types=["strong", "EM"],
    formalism="canonical-helicity",
    particle_db=particle_db,
    max_angular_momentum=3,
    max_spin_magnitude=3,
    mass_conservation_factor=0,
)

In [ ]:
for state_id, particle in reaction_strong_EM.initial_state.items():
    print(f"ID {state_id}:", particle.name)
for state_id, particle in reaction_strong_EM.final_state.items():
    print(f"ID {state_id}:", particle.name)

In [ ]:
src = qrules.io.asdot(reaction_strong_EM, collapse_graphs=True)
graphviz.Source(src)

## Build Helicity Amplitude Models

In [ ]:
model_builder = ampform.get_builder(reaction_strong_EM)
model_builder.config.scalar_initial_state_mass = True
model_builder.config.stable_final_state_ids = 0, 1, 2

bw_builder = RelativisticBreitWignerBuilder(
    energy_dependent_width=False,
    form_factor=False,
)

for name in reaction_strong_EM.get_intermediate_particles().names:
    model_builder.dynamics.assign(name, bw_builder)
model = model_builder.formulate()

The first component of the full amplitude is shown below.
The other terms are in similar way of formualation and thus not showing explicitly here agin.

In [ ]:
(symbol, expr), *_ = model.amplitudes.items()
Math(aslatex({symbol: expr}, terms_per_line=1))

### Parameters and kinematics variables

The parameters of the helicity amplitude model::

In [ ]:
sorted_parameter_defaults = {
    symbol: model.parameter_defaults[symbol]
    for symbol in sorted(model.parameter_defaults, key=str)
}

src = aslatex(sorted_parameter_defaults)
Math(src)

In [ ]:
Math(aslatex(model.kinematic_variables))

## Generate phase space sample

In [ ]:
phsp_event = 1_000_000
rng = TFUniformRealNumberGenerator(seed=0)
phsp_generator = TFPhaseSpaceGenerator(
    initial_state_mass=reaction_strong_EM.initial_state[-1].mass,
    final_state_masses={i: p.mass for i, p in reaction_strong_EM.final_state.items()},
)
phsp_momenta = phsp_generator.generate(phsp_event, rng)

In [ ]:
p = np.array(list(phsp_momenta.values()))
p.shape

In [ ]:
p.sum(axis=0).round(decimals=14)

In [ ]:
E0 = p[0].T[0]
px0 = p[0].T[1]
py0 = p[0].T[2]
pz0 = p[0].T[3]
p0 = np.sqrt(px0**2 + py0**2 + pz0**2)

E1 = p[1].T[0]
px1 = p[1].T[1]
py1 = p[1].T[2]
pz1 = p[1].T[3]
p1 = np.sqrt(px1**2 + py1**2 + pz1**2)

E2 = p[2].T[0]
px2 = p[2].T[1]
py2 = p[2].T[2]
pz2 = p[2].T[3]
p2 = np.sqrt(px2**2 + py2**2 + pz2**2)

In [ ]:
phsp_momenta_frame = pd.DataFrame(
    {
        (k, label): np.transpose(v)[i]
        for k, v in phsp_momenta.items()
        for i, label in enumerate(["E", "px", "py", "pz"])
    }
)

## Visualization

In [ ]:
unfolded_expression = model.expression.doit()
intensity_func = create_parametrized_function(
    expression=unfolded_expression,
    parameters=model.parameter_defaults,
    backend="numpy",
)

In [ ]:
helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="jax"
)

In [ ]:
phsp = helicity_transformer(phsp_momenta)
list(phsp)

In [ ]:
phsp_frame = pd.DataFrame(phsp)

In [ ]:
# %config InlineBackend.figure_formats = ['svg']

fig, ax = plt.subplots()
hist = ax.hist2d(
    phsp["m_01"].real ** 2,
    phsp["m_12"].real ** 2,
    bins=200,
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
    weights=intensity_func(phsp),
)
ax.set_title("Model-weighted Phase space Dalitz Plot")
ax.set_xlabel(R"$m^2(\eta \pi^0)\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$m^2(\pi^0 p)\;\left[\mathrm{GeV}^2\right]$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt

resonances = sorted(
    reaction_strong_EM.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(
    np.real(phsp_frame["m_01"]),
    bins=100,
    alpha=0.5,
    density=True,
    # color = "orange",
    label="Weighted PHSP",
    weights=intensity_func(phsp),
)
ax.set_xlabel(R"$m(\eta \pi^0)$ [GeV]")
ax.axvline(
    x=resonances[1].mass, linestyle="dotted", label=resonances[1].name, color="black"
)
ax.legend()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt

resonances = sorted(
    reaction_strong_EM.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(
    np.real(phsp_frame["m_12"]),
    bins=100,
    alpha=0.5,
    density=True,
    # color = colors[0],
    label="Weighted PHSP",
    weights=intensity_func(phsp),
)

ax.set_xlabel(R"$m(\pi^0 p)$ [GeV]")
ax.axvline(
    x=resonances[0].mass, linestyle="dotted", label=resonances[0].name, color=colors[0]
)
ax.axvline(
    x=resonances[2].mass, linestyle="dotted", label=resonances[2].name, color=colors[2]
)
ax.legend()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import matplotlib.pyplot as plt

resonances = sorted(
    reaction_strong_EM.get_intermediate_particles(),
    key=lambda p: p.mass,
)
evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [plt.cm.rainbow(x) for x in evenly_spaced_interval]
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(
    np.real(phsp_frame["m_02"]),
    bins=100,
    alpha=0.5,
    density=True,
    # color = colors[2],
    label="Weighted PHSP",
    weights=intensity_func(phsp),
)
ax.set_xlabel(R"$m(\eta p)$ [GeV]")
ax.axvline(
    x=resonances[2].mass, linestyle="dotted", label=resonances[2].name, color=colors[2]
)
ax.legend()
plt.show()